<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/Module_04_change_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 4: Change Detection

## Introduction

Many earth observation datasets are available at regular intervals over long periods of time. This enables us to detect changes on the Earth’s surface. Change detection technique in remote sensing fall in the following categories

1. Single Band Change: Measuring change in a single band image or a spectral index using a threshold
2. Multi Band Change: Measuring spectral distance and spectral angle between two multiband images
3. Classification of Change: One-pass classification using stacked image containing bands from before and after an event
4. Post Classification Comparison: Comparing two classified images and computing class transitions


In [20]:
import geemap
import ee

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Spectral Index Change

Many types of change can be detected by measuring the change in a spectral index and applying a threshold. This technique is suitable when there is a suitable spectral index is available for the type of change you are interested in detecting.

Here we apply this technique to map the extent and severity of a forest fire.

**Case study: Explore the forest burn in Sonitpur district, Assam**


In [21]:
adm = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
aoi = adm.filter(ee.Filter.eq('ADM2_NAME', "Sonitpur"))
geometry = aoi.geometry()

### Plotting the SWIR bands from sentinel

In [22]:
fireStart = ee.Date('2019-03-13')
fireEnd = ee.Date('2019-03-15')

## Load sentinel multispectral surface reflectance data
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
filtered = s2.filter(ee.Filter.bounds(geometry)).select('B.*')

## Clean the images by masking the cloud
csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
csPlusBands = csPlus.first().bandNames()

## Link the sentinel images with cloud score+ images
filtered_with_cs_plus = filtered.linkCollection(csPlus, csPlusBands)

def maskLowQA(image):
    qaBand = 'cs'
    clearThreshold = 0.5
    mask = image.select(qaBand).gte(clearThreshold)
    return image.updateMask(mask)


clean_s2 = filtered_with_cs_plus.map(maskLowQA)

## Filter the images for before and after forest fire, then create the composites
before = clean_s2.filter(ee.Filter.date(fireStart.advance(-2, 'month'),fireStart)).median()
after = clean_s2.filter(ee.Filter.date(fireEnd, fireEnd.advance(1,'month'))).median()

## Freshly burnt regions appeat bright in SWIR-bands
## Use a False Color Visualization
swirVis = {
    'min': 0,
    'max': 3000,
    'bands': ['B12', 'B8', 'B4']
}

## Create map and plot the results
Map = geemap.Map()
Map.addLayer(before.clip(geometry), swirVis, name='Before')
Map.addLayer(after.clip(geometry), swirVis, name='After')
Map.centerObject(geometry, zoom=11)
Map

Map(center=[26.76086472764834, 92.93095436904682], controls=(WidgetControl(options=['position', 'transparent_b…

### Plotting the normalized burnt ratio (NBR)

The Normalized Burn Ratio (NBR) is an index that is designed to highlight burnt vegetation areas. We compute the NBR for before and after images. Then we apply a suitable threshold to find burnt areas.


Interpreting the change in NBR AKA dNBR

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTPwwEWehWSioS6wKl0LQxueJzfOg4W4zHdE429L8dr&s)

In [27]:
## Calculate the NBR
def addNBR(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename(['nbr'])
    return image.addBands(nbr)


beforeNBR = addNBR(before).select('nbr')
afterNBR = addNBR(after).select('nbr')

nbrVis = {
    'min': -0.5,
    'max': 0.5,
    'palette': ['white', 'black']
}

Map.addLayer(beforeNBR.clip(geometry), nbrVis, name="NBR Before")
Map.addLayer(afterNBR.clip(geometry), nbrVis, name='NBR After')

## Calculate the change in NBR
change = beforeNBR.subtract(afterNBR)
threshold = 0.3

burned = change.gt(threshold)
dnbrPalette = ['green', 'yellow', 'orange', 'red', 'magenta']
Map.addLayer(change.clip(geometry), {'min':0, 'max':1, 'palette': dnbrPalette}, name='Burned')
Map

Map(bottom=111125.0, center=[26.773813330666446, 93.23547363281251], controls=(WidgetControl(options=['positio…

## Spectral distance change

When you want to detect changes from multi-band images, a useful technique is to compute the Spectral Distance and Spectral Angle between the two images. Pixels that exhibit a large change will have a larger distance compared to those that did not change. This technique is particularly useful when there are no suitable index to detect the change. It can be applied to detect change after natural disasters or human conflicts.

In [33]:
Map = geemap.Map()

## Use the spectralDistance() function to get spectral distance measures
## Use the metric 'Spectral Angle Mapper (SAM)
## The result is the spectral angle in radians

angle = after.spectralDistance(before, 'sam').rename('SAM')
Map.addLayer(angle.clip(geometry), {'min':0, 'max':1, 'palette': ['white', 'purple'], 'name': 'Spectral Angle'})

## Use the metric 'Squared Euclidian Distance (SED)
## Take square to get the euclidian distance
sed = after.spectralDistance(before, 'sed').rename('SED')
distance = sed.sqrt()
Map.addLayer(distance.clip(geometry), {'min':0, 'max':1500, 'palette': ['white', 'red'], 'name': 'Spectral Distance'})

Map.centerObject(geometry, zoom=11)
Map

Map(center=[26.76086472764834, 92.93095436904682], controls=(WidgetControl(options=['position', 'transparent_b…

Exercise
Inspect the angle image and find a suitable threshold
that signifies damage after the landslides

Apply the threshold and create a new image showing landslides

Display the results

Hint: Use the .gt() method to apply the threshold

In [36]:
critical_angle = angle.gt(0.2)

Map = geemap.Map()
Map.addLayer(critical_angle.clip(geometry), {'min':0, 'max':1, 'palette': ['white', 'red'], 'name': 'Critical SAM'})
Map.centerObject(geometry, zoom=11)
Map

Map(center=[26.76086472764834, 92.93095436904682], controls=(WidgetControl(options=['position', 'transparent_b…

## Direct Classification of Change

